In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from common import *

In [3]:
import argparse
parser = argparse.ArgumentParser(description='Run Diagnosis experiments')
parser.add_argument("--data_dir", type=str, required=True)
parser.add_argument('--display', dest='display', action='store_true')
parser.add_argument("--output_dir", type=str)
parser.add_argument("--mock", dest='mock', action='store_true')

args = parser.parse_args(['--data_dir=.', '--output_dir=outputs/', '--display'])

In [4]:
from PatientVec.models.Vanilla import ClassificationTrainer as BasicCT
from PatientVec.models.Hierarchical import ClassificationTrainer as HierCT
from PatientVec.trainer import Trainer, Evaluator
from PatientVec.Experiments.modifiable_config_exp import vanilla_configs, attention_configs, hierarchical_configs, structured_configs

from PatientVec.Experiments.hyperparam_exps import *
from PatientVec.models.baselines.LR import LR, LDA

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [5]:
from dataloaders import hip_dataset, knee_dataset

In [ ]:
for yr in [1, 2, 3] :
    data = hip_dataset(args, yr=yr)
#     train_data, dev_data = get_basic_data(data, structured=False, truncate=100)

#     lr = LR({'vocab' : data.vocab, 'stop_words' : True, 'exp_name' : data.name, 'type' : 'classifier', 'norm' : 'l2'})
#     lr.train(train_data)
#     lr.evaluate(dev_data, save_results=True)
#     try :
#         lr.print_all_features(n=30)
#     except :
#         pass
        
#     print('=' * 200)

    train_data, dev_data = get_basic_data(data, structured=False, truncate=98)

    for e in attention_configs :
        config = e(data, structured=False, args=args)
        if args.output_dir is not None :
            config['exp_config']['basepath'] = args.output_dir
        config['training_config']['common']['bsize'] = 8
        config['training_config']['common']['class_weight'] = False
        config['training_config']['common']['balanced'] = True
        config['exp_config']['exp_name'] += '+Balanced'
        config['training_config']['type'] = 'RMSprop'
        print(config)

        trainer = Trainer(BasicCT, config, _type=data.metrics_type, display_metrics=True)
        trainer.train(train_data, dev_data, n_iters=15, save_on_metric=data.save_on_metric)

        evaluator = Evaluator(BasicCT, trainer.model.dirname, _type=data.metrics_type, display_metrics=True)
        _ = evaluator.evaluate(dev_data, save_results=True)
        print('-'*300)

    for e in vanilla_configs :
        config = e(data, structured=False, args=args)
        if args.output_dir is not None :
            config['exp_config']['basepath'] = args.output_dir
        config['training_config']['common']['bsize'] = 8
        config['training_config']['common']['class_weight'] = False
        config['training_config']['common']['balanced'] = True
        config['exp_config']['exp_name'] += '+Balanced'
        print(config)

        trainer = Trainer(BasicCT, config, _type=data.metrics_type, display_metrics=True)
        trainer.train(train_data, dev_data, n_iters=15, save_on_metric=data.save_on_metric)

        evaluator = Evaluator(BasicCT, trainer.model.dirname, _type=data.metrics_type, display_metrics=True)
        _ = evaluator.evaluate(dev_data, save_results=True)
        print('-'*300)
        
    print('='*500)

In [ ]:
for yr in [1, 2, 3] :
    data = hip_dataset(args, yr=yr)
#     train_data, dev_data = get_basic_data(data, structured=False, truncate=100)

#     lr = LR({'vocab' : data.vocab, 'stop_words' : True, 'exp_name' : data.name, 'type' : 'classifier', 'norm' : 'l2'})
#     lr.train(train_data)
#     lr.evaluate(dev_data, save_results=True)
#     try :
#         lr.print_all_features(n=30)
#     except :
#         pass
        
#     print('=' * 200)

    train_data, dev_data = get_basic_data(data, structured=False, truncate=98)

    for e in vanilla_configs :
        config = e(data, structured=False, args=args)
        if args.output_dir is not None :
            config['exp_config']['basepath'] = args.output_dir
        config['training_config']['common']['bsize'] = 8
        config['model']['predictor']['replicate'] = True
        config['model']['predictor']['alpha'] = 0.3
        config['exp_config']['exp_name'] += '+Replicate'
        config['training_config']['type'] = 'RMSprop'
        print(config)

        trainer = Trainer(BasicCT, config, _type=data.metrics_type, display_metrics=True)
        trainer.train(train_data, dev_data, n_iters=15, save_on_metric=data.save_on_metric)

        evaluator = Evaluator(BasicCT, trainer.model.dirname, _type=data.metrics_type, display_metrics=True)
        _ = evaluator.evaluate(dev_data, save_results=True)
        print('-'*300)
        
    print('='*500)

In [11]:
for yr in [1, 2, 3] :
    data = hip_dataset(args, yr=yr)
    args.norm = 'l2'
    args.bow_type = 'bow'
    args.structured = False
    vector_experiments(data, args)

INFO - 2019-03-12 15:02:29,047 - Reading Structured data ...
INFO - 2019-03-12 15:02:29,174 - Reading Notes ...
INFO - 2019-03-12 15:02:29,859 - Stratifying ...


Pos Percentage [0.10839746]


INFO - 2019-03-12 15:02:30,317 - Maximum Sentence Length 585530.000000 , 100 percentile length 585530.000000 ... 
INFO - 2019-03-12 15:02:31,407 - Truncated all ...
INFO - 2019-03-12 15:02:31,497 - Maximum Sentence Length 65362.000000 , 100 percentile length 65362.000000 ... 


Pos Percentage [0.08990536]


INFO - 2019-03-12 15:02:32,489 - Truncated all ...


...

,0.0,1.0,micro avg,macro avg,weighted avg
f1-score,0.940,0.378,0.891,0.659,0.890
precision,0.938,0.389,0.891,0.663,0.889
recall,0.943,0.368,0.891,0.656,0.891
support,577.000,57.000,634.000,634.000,634.000


INFO - 2019-03-12 15:12:14,414 - instantiating class <class 'PatientVec.models.Model.Model'> from params {'type': 'vec_classifier', 'decoder': {'num_layers': 2, 'hidden_dims': [256, 1], 'activations': ['tanh', 'linear'], 'dropout': [0.2, 0.0], 'input_dim': 24612}, 'predictor': {'type': 'binary'}, 'structured': {'use_structured': False}, 'reg': {'type': 'l2', 'weight': 0.5}} and extras {}
INFO - 2019-03-12 15:12:14,414 - type = vec_classifier
INFO - 2019-03-12 15:12:14,415 - decoder.input_dim = 24612
INFO - 2019-03-12 15:12:14,415 - decoder.num_layers = 2
INFO - 2019-03-12 15:12:14,415 - decoder.hidden_dims = [256, 1]
INFO - 2019-03-12 15:12:14,415 - decoder.activations = ['tanh', 'linear']
INFO - 2019-03-12 15:12:14,416 - decoder.dropout = [0.2, 0.0]
INFO - 2019-03-12 15:12:14,442 - instantiating class <class 'PatientVec.models.modules.Predictor.Predictor'> from params {'type': 'binary'} and extras {}
INFO - 2019-03-12 15:12:14,442 - predictor.type = binary
INFO - 2019-03-12 15:12:14,4

Model not saved on  roc_auc 0.7540971145367752




{'accuracy': 0.807570977917981, 'roc_auc': 0.817857034266776, 'pr_auc': 0.33795297417020287}


,0.0,1.0,micro avg,macro avg,weighted avg
f1-score,0.887,0.358,0.808,0.622,0.839
precision,0.954,0.256,0.808,0.605,0.891
recall,0.828,0.596,0.808,0.712,0.808
support,577.000,57.000,634.000,634.000,634.000


Saving Models
==============

In [ ]:
from common import generate_latex_tables
keys_to_use = ['accuracy', 'roc_auc', 'pr_auc']
for yr in [1, 2, 3] :
    data = knee_dataset(args, yr=yr)
    generate_latex_tables(data, keys_to_use)

In [ ]:
for model in ['Attention', 'baselines', 'Basic'] :
    yr_df = {}
    for yr in [1, 2, 3] :
        df = pd.read_csv('Text-encoding-EHR/results/HipSurgery_' + str(yr) + '/' + model + '.csv')
        df.index = df['Method']
        df = df.drop(columns=['Method'])
        yr_df[yr] = df
    yr_df = pd.concat(yr_df.values(), axis=1, keys=yr_df.keys())
    yr_df.columns = yr_df.columns.swaplevel(0, 1)
    yr_df.sort_index(axis=1, level=0, inplace=True)
    display(HTML(yr_df.to_html()))

In [ ]:
for model in ['Attention', 'baselines', 'Basic'] :
    yr_df = {}
    for yr in [1, 2, 3] :
        df = pd.read_csv('Text-encoding-EHR/results/KneeSurgery_' + str(yr) + '/' + model + '.csv')
        df.index = df['Method']
        df = df.drop(columns=['Method'])
        yr_df[yr] = df
    yr_df = pd.concat(yr_df.values(), axis=1, keys=yr_df.keys())
    yr_df.columns = yr_df.columns.swaplevel(0, 1)
    yr_df.sort_index(axis=1, level=0, inplace=True)
    display(HTML(yr_df.to_html()))